# Forms

> Form generation components for settings interfaces

In [ ]:
#| default_exp components.forms

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional, List
from fasthtml.common import *
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_styles
from cjm_fasthtml_daisyui.components.data_display.card import card_actions
from cjm_fasthtml_daisyui.utilities.semantic_colors import border_dui
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_jsonschema.generators.form import generate_form_ui
from cjm_fasthtml_settings.core.html_ids import HtmlIds

## Settings Form

In [ ]:
#| export
def create_settings_form(
    schema: Dict[str, Any],  # JSON schema for the form
    values: Dict[str, Any],  # Current values for the form fields
    post_url: str,  # URL for form submission
    reset_url: str  # URL for resetting form to defaults
) -> Form:  # Form element with settings and action buttons
    """Create a settings form with action buttons.
    
    Generates a form using cjm-fasthtml-jsonschema based on the provided schema,
    with Save and Reset buttons.
    """

    # Build button attributes for Save button
    save_button_attrs = {
        "type": "submit",
        "cls": combine_classes(btn, btn_colors.primary)
    }

    # Add custom onclick handler if defined in schema
    if "onclick_save" in schema:
        save_button_attrs["onclick"] = schema["onclick_save"]

    # Build button attributes for Reset button
    reset_button_attrs = {
        "type": "button",
        "hx_get": reset_url,
        "hx_target": HtmlIds.as_selector(HtmlIds.SETTINGS_CONTENT),
        "hx_swap": "innerHTML",
        "cls": combine_classes(btn, btn_styles.ghost, m.l(2))
    }

    # Add custom onclick handler if defined in schema
    if "onclick_reset" in schema:
        reset_button_attrs["onclick"] = schema["onclick_reset"]

    return Form(
        generate_form_ui(
            schema=schema,
            values=values,
            show_title=True,
            show_description=True,
            compact=False,
            card_wrapper=True
        ),

        # Form actions
        Div(
            Button(
                "Save Configuration",
                **save_button_attrs
            ),
            Button(
                "Reset to Defaults",
                **reset_button_attrs
            ),
            cls=combine_classes(
                card_actions,
                m.t(6),
                border_dui.base_300,
            )
        ),

        # Form submission
        hx_post=post_url,
        hx_target=HtmlIds.as_selector(HtmlIds.SETTINGS_CONTENT),
        hx_swap="innerHTML"
    )

In [ ]:
# Example: Create a settings form
from cjm_fasthtml_settings.core.config import get_app_config_schema
from cjm_fasthtml_settings.core.utils import get_default_values_from_schema

schema = get_app_config_schema(app_title="Test App", include_theme=False)
values = get_default_values_from_schema(schema)

form = create_settings_form(
    schema=schema,
    values=values,
    post_url="/settings/save/general",
    reset_url="/settings/reset/general"
)

form

```html
<form enctype="multipart/form-data" hx-post="/settings/save/general" hx-swap="innerHTML" hx-target="#settings-content">  <div class="card bg-base-100 shadow-lg w-full max-w-4xl">
    <div class="card-body">
      <h2 class="card-title">Application Configuration</h2>
      <div class="text-base-content text-sm mb-6">General application settings and configuration</div>
      <div class="font-semibold text-base-content mb-3">Required Fields</div>
      <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">App Title</span><span class="text-error font-bold"> *</span></label>        <input type="text" name="app_title" value="Test App" placeholder="Default: Test App" required minlength="1" maxlength="100" id="field-app_title" class="input input-md w-full">
        <p class="text-sm text-base-content mt-1 mb-2">Title displayed in the navbar and browser tab</p>
      </div>
      <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Config Dir</span><span class="text-error font-bold"> *</span></label>        <input type="text" name="config_dir" value="configs" placeholder="Default: configs" required pattern='^[^&lt;&gt;:"|?*]+$' id="field-config_dir" class="input input-md w-full validator">
        <p class="text-sm text-base-content mt-1 mb-2">Directory where configuration files are stored</p>
      </div>
<hr class="my-6">      <div class="font-semibold text-base-content mb-3">Optional Fields</div>
      <div class="mb-4">
<label class="label flex items-center cursor-pointer">          <input type="checkbox" name="auto_open_browser" checked id="field-auto_open_browser" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Auto Open Browser</span></label>        <p class="text-sm text-base-content mt-1 mb-2">Automatically open browser when starting the application</p>
      </div>
      <div class="mb-4">
<label class="label flex items-center cursor-pointer">          <input type="checkbox" name="debug_mode" id="field-debug_mode" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Debug Mode</span></label>        <p class="text-sm text-base-content mt-1 mb-2">Enable debug mode for development</p>
      </div>
      <div class="mb-4">
        <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Max Upload Size Mb</span></label><span class="text-xs text-base-content ml-2">Min: 1 • Max: 10240</span>        </div>
        <input type="number" name="max_upload_size_mb" value="100" min="1" max="10240" step="any" id="field-max_upload_size_mb" class="input input-md w-full">
        <p class="text-sm text-base-content mt-1 mb-2">Maximum file upload size in megabytes</p>
      </div>
      <div class="mb-4">
<label class="label flex items-center cursor-pointer">          <input type="checkbox" name="reload_on_change" id="field-reload_on_change" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Reload On Change</span></label>        <p class="text-sm text-base-content mt-1 mb-2">Automatically reload the application when files change (development only)</p>
      </div>
      <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Server Host</span></label><select name="server_host" id="field-server_host" class="select select-md w-full"><option>Select an option...</option><option value="0.0.0.0" selected>0.0.0.0</option><option value="127.0.0.1">127.0.0.1</option><option value="localhost">localhost</option></select>        <p class="text-sm text-base-content mt-1 mb-2">Host address for the web server</p>
      </div>
      <div class="mb-4">
        <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Server Port</span></label><span class="text-xs text-base-content ml-2">Min: 1024 • Max: 65535</span>        </div>
        <input type="number" name="server_port" value="5000" min="1024" max="65535" step="1" id="field-server_port" class="input input-md w-full">
        <p class="text-sm text-base-content mt-1 mb-2">Port number for the web server</p>
      </div>
      <div class="mb-4">
        <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Session Timeout Minutes</span></label><span class="text-xs text-base-content ml-2">Min: 0 • Max: 1440</span>        </div>
        <input type="number" name="session_timeout_minutes" value="60" min="0" max="1440" step="1" id="field-session_timeout_minutes" class="input input-md w-full">
        <p class="text-sm text-base-content mt-1 mb-2">Session timeout in minutes (0 for no timeout)</p>
      </div>
    </div>
  </div>
  <div class="card-actions mt-6 border-base-300">
<button type="submit" class="btn btn-primary">Save Configuration</button><button type="button" hx-get="/settings/reset/general" hx-swap="innerHTML" hx-target="#settings-content" class="btn btn-ghost ml-2">Reset to Defaults</button>  </div>
</form>
```

## Settings Form Container

In [ ]:
#| export
def create_settings_form_container(
    schema: Dict[str, Any],  # JSON schema for the form
    values: Dict[str, Any],  # Current values for the form fields
    post_url: str,  # URL for form submission
    reset_url: str,  # URL for resetting form to defaults
    alert_message: Optional[Any] = None,  # Optional alert element to display
    use_alert_container: bool = False  # If True, add empty alert-container div
) -> Div:  # Div containing the alert (if any) and the settings form
    """Create a container with optional alert and settings form.
    
    This is useful for wrapping a settings form with an alert area that can
    display success/error messages.
    """
    children = []

    # Add alert or alert container
    if alert_message:
        children.append(alert_message)
    elif use_alert_container:
        children.append(Div(id=HtmlIds.ALERT_CONTAINER))

    # Add the form
    children.append(
        create_settings_form(
            schema=schema,
            values=values,
            post_url=post_url,
            reset_url=reset_url
        )
    )

    return Div(*children)

In [ ]:
# Example: Create form with alert container
from cjm_fasthtml_settings.core.config import get_app_config_schema
from cjm_fasthtml_settings.core.utils import get_default_values_from_schema

schema = get_app_config_schema(app_title="Test App", include_theme=False)
values = get_default_values_from_schema(schema)

container = create_settings_form_container(
    schema=schema,
    values=values,
    post_url="/settings/save/general",
    reset_url="/settings/reset/general",
    use_alert_container=True
)

container

```html
<div>
  <div id="alert-container"></div>
<form enctype="multipart/form-data" hx-post="/settings/save/general" hx-swap="innerHTML" hx-target="#settings-content">    <div class="card bg-base-100 shadow-lg w-full max-w-4xl">
      <div class="card-body">
        <h2 class="card-title">Application Configuration</h2>
        <div class="text-base-content text-sm mb-6">General application settings and configuration</div>
        <div class="font-semibold text-base-content mb-3">Required Fields</div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">App Title</span><span class="text-error font-bold"> *</span></label>          <input type="text" name="app_title" value="Test App" placeholder="Default: Test App" required minlength="1" maxlength="100" id="field-app_title" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Title displayed in the navbar and browser tab</p>
        </div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Config Dir</span><span class="text-error font-bold"> *</span></label>          <input type="text" name="config_dir" value="configs" placeholder="Default: configs" required pattern='^[^&lt;&gt;:"|?*]+$' id="field-config_dir" class="input input-md w-full validator">
          <p class="text-sm text-base-content mt-1 mb-2">Directory where configuration files are stored</p>
        </div>
<hr class="my-6">        <div class="font-semibold text-base-content mb-3">Optional Fields</div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="auto_open_browser" checked id="field-auto_open_browser" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Auto Open Browser</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Automatically open browser when starting the application</p>
        </div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="debug_mode" id="field-debug_mode" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Debug Mode</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Enable debug mode for development</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Max Upload Size Mb</span></label><span class="text-xs text-base-content ml-2">Min: 1 • Max: 10240</span>          </div>
          <input type="number" name="max_upload_size_mb" value="100" min="1" max="10240" step="any" id="field-max_upload_size_mb" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Maximum file upload size in megabytes</p>
        </div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="reload_on_change" id="field-reload_on_change" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Reload On Change</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Automatically reload the application when files change (development only)</p>
        </div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Server Host</span></label><select name="server_host" id="field-server_host" class="select select-md w-full"><option>Select an option...</option><option value="0.0.0.0" selected>0.0.0.0</option><option value="127.0.0.1">127.0.0.1</option><option value="localhost">localhost</option></select>          <p class="text-sm text-base-content mt-1 mb-2">Host address for the web server</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Server Port</span></label><span class="text-xs text-base-content ml-2">Min: 1024 • Max: 65535</span>          </div>
          <input type="number" name="server_port" value="5000" min="1024" max="65535" step="1" id="field-server_port" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Port number for the web server</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Session Timeout Minutes</span></label><span class="text-xs text-base-content ml-2">Min: 0 • Max: 1440</span>          </div>
          <input type="number" name="session_timeout_minutes" value="60" min="0" max="1440" step="1" id="field-session_timeout_minutes" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Session timeout in minutes (0 for no timeout)</p>
        </div>
      </div>
    </div>
    <div class="card-actions mt-6 border-base-300">
<button type="submit" class="btn btn-primary">Save Configuration</button><button type="button" hx-get="/settings/reset/general" hx-swap="innerHTML" hx-target="#settings-content" class="btn btn-ghost ml-2">Reset to Defaults</button>    </div>
</form></div>

```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()